# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [ ]:
from openai import OpenAI

client = OpenAI(base_url='https://api.xiaoai.plus/v1')

In [1]:
from openai import OpenAI

client = OpenAI(base_url='https://api.xiaoai.plus/v1')

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

: 

In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [4]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [5]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默风格的图片，展示了两只经过搞怪编辑的狗：一只被想象成肌肉非常发达的“健身版本”，一只则是常规形态的“正常版本”。左边的狗被编辑成拥有强健肌肉的人形身体，给人以强烈的视觉冲击；右边的狗则呈现较为自然的坐姿和体态。

图中包含一些文字，左边的文字大意为描述健身版狗狗的特征，包括可以一口吃掉大米饭团、有超强的力量等。右边的文字则描述了常规狗狗更为温和和实际的特征，如对小动物表示友好、不擅长表达悲伤等。整幅图表达了一种幽默感，通过夸张对比强调体型和行为的差异。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [2]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url='https://api.xiaoai.plus/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图展示了1980年到2020年间，美国、中国、日本和德国的国内生产总值（GDP）比较。每个国家的GDP以万亿美元为单位标出。从图中可以看出：

- **美国（蓝线）**：GDP持续增长，从1980年的约3万亿美金增长到2020年的超过20万亿美金。
- **中国（红线）**：自1980年起，GDP增长非常显著，从不到1万亿美金增长至接近15万亿美金。
- **日本（紫线）**：1980年至1995年GDP增长明显，之后增长放缓，并在2005年至2015年间保持相对平稳。
- **德国（绿线）**：GDP增长较为平稳，从1980年的不到1万亿美金增长至2020年的约4万亿美金。

总体而言，图表清晰地表示了这四个经济体在过去四十年间的经济增长情况，其中中国的增长尤其引人注目，显示了其经济的迅速崛起。美国则保持了其全球经济领导地位的增长趋势。日本在90年代后增长放缓，而德国则显示出稳定的增长模式。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

![](./images/question1.png)

In [11]:
content = query_base64_image_description("./images/question1.png")
print(content)

这张图片上是一张纸，上面用黑色笔迹写着中文，内容如下：

“面试题
你1000米跑，沿一种动物走廊跑，你猜何动物走廊跑过？”

这是一个谜语式的面试题，题目问的是在某种动物的走廊上跑1000米，询问这是哪种动物的走廊。这种题目常用于考查应聘者的思维方式、创意或解决问题的能力。不过，没有提供更多信息或上下文，这个问题的答案不是很明确，可能需要依赖更多的背景信息或是具体指导才能给出答案。


![](./images/question2.png)

In [12]:
content = query_base64_image_description("./images/question2.png")
print(content)

这张图显示的是一份简短的通知内容。文字说明了一些有关时间的安排：

1. “凡是个小组成员的截稿、结了个城一堵，嘴唇后不个人、住9人每脖一组。嘴唇后不个人；落11人每脖一组。嘴唇后不个人。” 这句话因为排版或印刷错误，显得语句不通，不容易理解，似乎有一些文字缺失或者是错误。

2. 然后提到，“期会一起解放战的小朋友至章一人。”

结合上下文，此通知可能是关于一些小组成员的时间安排和某种活动或任务的分配，但由于文本错误，具体内容难以完全明确解释。


![](./images/gpt-4v.jpg)

In [3]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

这是一张关于计算机网络中NAT（网络地址转换）技术的笔记。主要内容如下：

1. **NAT的基本描述**：
    - NAT允许多个设备共享一个公共IP地址来访问互联网，这对于IP地址不足的情况非常有用。
    - 使用`ipconfig`命令可查看网络配置信息，包括IP地址。

2. **私有地址和公网地址**：
    - 强调了在局域网使用的私有IP与公网IP的区别。私有IP地址段通常包括10.0.0.0/8, 172.16.0.0/12, 192.168.0.0/16，这些地址不能直接访问互联网。

3. **NAT操作过程**：
    - 描述了NAT如何将内网地址转换为公网地址来进行互联网访问，并且如何将互联网的数据路由回内部网络指定的设备。
    - 提到NAT可为公司节省显著的公网IP地址成本，提高安全性。

4. **IPV6介绍**：
    - 说明了随着IPv4地址的逐渐耗尽，IPv6的推出为网络设备提供了更多的IP地址资源。

这些笔记以中文写成，详细解释了网络中NAT的功能和重要性，以及IPv6在现代网络中的作用。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [4]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这是一张关于计算机网络中NAT（网络地址转换）技术的笔记。主要内容如下：

1. **NAT的基本描述**：
    - NAT允许多个设备共享一个公共IP地址来访问互联网，这对于IP地址不足的情况非常有用。
    - 使用`ipconfig`命令可查看网络配置信息，包括IP地址。

2. **私有地址和公网地址**：
    - 强调了在局域网使用的私有IP与公网IP的区别。私有IP地址段通常包括10.0.0.0/8, 172.16.0.0/12, 192.168.0.0/16，这些地址不能直接访问互联网。

3. **NAT操作过程**：
    - 描述了NAT如何将内网地址转换为公网地址来进行互联网访问，并且如何将互联网的数据路由回内部网络指定的设备。
    - 提到NAT可为公司节省显著的公网IP地址成本，提高安全性。

4. **IPV6介绍**：
    - 说明了随着IPv4地址的逐渐耗尽，IPv6的推出为网络设备提供了更多的IP地址资源。

这些笔记以中文写成，详细解释了网络中NAT的功能和重要性，以及IPv6在现代网络中的作用。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。